In [2]:
import os
import os.path as osp
import argparse
import torch
import time
from torch.utils.data import DataLoader
from mmengine.utils import mkdir_or_exist
from mmengine.config import Config, DictAction
from mmengine.logging import MMLogger

import matplotlib.pyplot as plt
import torch.nn.functional as F
import torchvision.transforms as T
from PIL import Image
from kornia.filters import gaussian_blur2d
from omegaconf import DictConfig, OmegaConf
from pytorch_lightning import seed_everything
from torch.utils.data import DataLoader, Subset
from torch.utils.tensorboard import SummaryWriter
from torchmetrics.functional.regression import explained_variance
from tqdm import tqdm

from estimator.utils import RunnerInfo, setup_env, log_env, fix_random_seed
from estimator.models.builder import build_model
from estimator.datasets.builder import build_dataset
from estimator.tester import Tester
from estimator.models.featup import featup
from mmengine import print_log

from external.featup.datasets.JitteredImage import JitteredImage, apply_jitter
from external.featup.datasets.util import get_dataset, SlicedDataset
from external.featup.downsamplers import SimpleDownsampler, AttentionDownsampler
from external.featup.featurizers.util import get_featurizer
from external.featup.layers import ImplicitFeaturizer, MinMaxScaler, ChannelNorm
from external.featup.losses import total_variation
from external.featup.util import (norm as reg_norm, unnorm as reg_unorm, generate_subset,
                         midas_norm, midas_unnorm, pca, PCAUnprojector, prep_image)

torch.multiprocessing.set_sharing_strategy('file_system')